In [ ]:
# Imports and file paths
# No CSV export needed — data goes straight from binary .nfr to DataFrame to plots
import pandas as pd
import plotly.express as px
import plotly.io as pio
from compile import compile_csv
from decode import decode_frame
from main import read_frames_from_file

# Use the VS Code renderer to avoid nbformat mime rendering issues
pio.renderers.default = "vscode"

DBC_PATH = "NFR26CANDBC.csv"
DATA_PATH = "testData/2-17-26/LOG_0001.NFR"

## 1. Compile DBC and decode frames

`compile_csv()` builds a decode table from the DBC CSV (signal definitions: bit positions, scales, offsets).
`read_frames_from_file()` reads raw 18-byte frames from the binary `.nfr` file.
`decode_frame()` extracts signal values from each frame's bytes using bit masking and scale/offset math.

Each decoded signal becomes a row in an in-memory DataFrame — no intermediate CSV file is written.

# Signal Graphing

This notebook reads CAN signals directly from binary `.nfr` log files and graphs them interactively using Plotly. No intermediate CSV is needed.

**Data flow:** DBC CSV (signal definitions) + `.nfr` binary (raw CAN frames) → decode in memory → pandas DataFrame → interactive plots

In [12]:
# Step 1: compile_csv() parses DBC CSV into a decode table (frame_id -> message/signal definitions)
# Step 2: read_frames_from_file() reads 18-byte LogFrames from binary .nfr (timestamp, id, dlc, data)
# Step 3: decode_frame() bit-masks each signal from the raw bytes, applies scale/offset
# Step 4: each signal value becomes a row in the DataFrame — no CSV written to disk
decode_table = compile_csv(DBC_PATH)

signal_units = {}
for msg in decode_table.values():
    for sig in msg.signals:
        signal_units[(msg.frame_id, sig.name)] = sig.unit or ""

rows = []
for timestamp, frame_id, data in read_frames_from_file(DATA_PATH):
    decoded = decode_frame(frame_id, data, decode_table)
    if not decoded:
        continue
    msg = decode_table[frame_id]
    for signal_name, value in decoded.items():
        unit = signal_units.get((frame_id, signal_name), "")
        rows.append({
            "signal_name": signal_name,
            "timestamp_ms": timestamp,
            "value": value,
            "message_id": f"0x{frame_id:X}",
            "message_name": msg.name,
            "unit": unit,
        })

df = pd.DataFrame(rows)

# Check if the file was empty or only contained a header with no frames
if df.empty:
    print(f"No signals decoded from {DATA_PATH} — file may be empty or contain only a header.")
else:
    print(f"{len(df)} signal readings from {df['signal_name'].nunique()} unique signals")
    df.head(10)

6174 signal readings from 95 unique signals


## 2. Plot helper

In [13]:
def plot_signal(sig_df, signal_name, message_name, message_id):
    unit = sig_df["unit"].iloc[0]
    y_label = f"{signal_name} ({unit})" if unit else signal_name

    fig = px.line(
        sig_df,
        x="timestamp_ms",
        y="value",
        title=f"{signal_name} — {message_name} ({message_id})",
        labels={"timestamp_ms": "Time (ms)", "value": y_label},
    )
    fig.show()

## 3. Graph all signals

Loops over every unique (message, signal) pair in the DataFrame and creates an interactive plot for each. Skips if data is empty.

In [14]:
# Loop over every unique (message, signal) pair and plot each one
# Skips entirely if no signals were decoded
if df.empty:
    print("No signals decoded.")
else:
    for (msg_name, sig_name), group in df.groupby(["message_name", "signal_name"]):
        msg_id = group["message_id"].iloc[0]
        plot_signal(group, sig_name, msg_name, msg_id)